# Set SageMaker environment

In [1]:
import sagemaker
import numpy as np
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel

sagemaker_session = sagemaker.Session()


role = 'arn:aws:iam::753384412675:role/service-role/AmazonSageMaker-ExecutionRole-20210810T104409' 

# Upload model artifacts to S3

In [2]:
# Path of the trained model artefact (tar.gz archive made by SageMaker estimator, from Yolo output (weights folder))
# This archive should contains the two trained model from Yolo : last.pt and best.pt
!cd /home/ubuntu/yolo5_train/yolov5/runs/train/exp5/weights && tar -czvf powercor-yolo5-pytorch-model.tar.gz *
model_artefact = 's3://asset-det-train-pipeline/yolo5/model_artefacts/powercor-yolo5-pytorch-model.tar.gz'
!aws s3 cp powercor-yolo5-pytorch-model.tar.gz model_artefacts


best.pt
last.pt
powercor-yolo5-pytorch-model.tar.gz
tar: powercor-yolo5-pytorch-model.tar.gz: file changed as we read it
/usr/lib/python3/dist-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import x509

usage: aws s3 cp <LocalPath> <S3Uri> or <S3Uri> <LocalPath> or <S3Uri> <S3Uri>
Error: Invalid argument type


In [3]:
# inference model image
inference_container_uri = '753384412675.dkr.ecr.ap-southeast-2.amazonaws.com/yolo5-inference:sagemaker'

# Runtime model definition as PyTorch model (Yolo v5 is based on PyTorch)

The local folder "code" will be embedded into this endpoint, to allow custom behavior of the container instance.

In this case we use a custom python class (ModelHandler) as a singleton to load the model and to manage inference request with this loaded model (please review code/model_handler.py file). This class is made to be compatible with the SageMaker model_server interface (https://github.com/aws/sagemaker-inference-toolkit)


In [17]:
# Create SageMaker model and deploy an endpoint
model = PyTorchModel(
    name=name_from_base('powercor-yolov5'),
    model_data=model_artefact,
    # entry_point='dockerd-entrypoint.py',
    entry_point='inference.py',
    role=role,
    source_dir='code',
    framework_version='1.8.0',
    py_version='py3',
    image_uri=inference_container_uri,
)

# Endpoint creation

The endpoint instance type can be customized depending on the desired hardware resource. "local" instance type allow to test the endpoint locally (docker/docker-compose are required)

In [15]:
#predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large')
# predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')
predictor = model.deploy(initial_instance_count=1, instance_type='local')

!

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/sagemaker/local/image.py", line 852, in run
    _stream_output(self.process)
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/sagemaker/local/image.py", line 914, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/sagemaker/local/image.py", line 857, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp5mscni0o/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 

In [16]:
# Print the name of newly created endpoint
print(predictor.endpoint_name)

powercor-yolov5-2022-05-12-14-24-16-528


## Local endpoint invoke with curl

```bash
curl -v -X POST -F "body=@/path/to/some-image.jpg" http://localhost:8080/models/model/invoke
```

## Sagemaker endpoint remote invoke :
(use the endpoint name printed out from the previous cell)

```bash
aws sagemaker-runtime invoke-endpoint --endpoint-name powercor-yolov5 --body fileb:///path/to/some-image.jpg --content-type multipart/form-data  >(cat)
```

# Endpoint removal

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint_name)